# Consumer Demand
Consider a consumer with an income of $I$, who is choosing between $n$ different goods.
We encode the prices of these goods as a vector $p \in \mathbb R^n$.
We assume that they have Cobb-Douglass preferences:

$$ u(x) = \sum_{i=1}^n \alpha_i \log(x_i) $$
where $\alpha$ is a vector of parameters satisfying $\sum \alpha_i = 1$.
This looks like a constrained optimization problem:
\begin{equation*}
\begin{aligned}
    \max_{x \in \R^n}\;\;   &  \sum_{i=1}^n \alpha_i \log(x_i) \\ 
    \text{s.t. } \;\;       &  p \cdot x \leq I \\ 
                            &  x_i \geq 0
\end{aligned}
\end{equation*}

This problem is nice because it has known solutions in closed form, so we can always go back and check our work!
If we take the Lagrangian for this problem, we get:
$$ \mathcal L = \sum_{i=1}^n \alpha_i \log(x_i) - \lambda (p \cdot x - I) $$
and a set of first order conditions 
\begin{align*}
    0 = {\partial \mathcal L \over \partial x_i}     &= {\alpha_i \over x_i} - \lambda p_i \\
    0 = {\partial \mathcal L \over \partial \lambda} &= p \cdot x - I 
\end{align*}

We can show that these imply:
$$ p_i x_i = {\alpha_i \over \lambda} 
\Rightarrow I = \sum_{i=1}^n p_i x_i 
= {1 \over \lambda} \sum_{i=1}^n \alpha_i 
= {1 \over \lambda} $$
So we obtain that 
$$ x_i = {\alpha_i \over p_i} I $$
The consumers spend a constant share of their income $\alpha_i$ on good $i$.
This should all look very familiar.

## The Plan
We are going to solve this problem in three different ways:
1. Sequential Quadratic Programming
2. Penalty Function
3. Augmented Lagrangian

In [9]:
using Optim
using ForwardDiff: gradient, hessian, jacobian
using Random
using LinearAlgebra
using TransformVariables

# Set the seed so this is reproducible
Random.seed!(1257)

# Setup our utility function
u(x, α) = sum(α[i] * log(x[i]) for i in eachindex(x, α))

# Make some test parameters
const p = rand(10); p[1] = 1          # random prices -- normalize 
const α = rand(10); α .*= 1/sum(α)     # ensure that α sum to 1

# We'll assume that income is 1 (to make things easy)


10-element Vector{Float64}:
 0.07714052044211762
 0.1588154045133561
 0.09263306228889596
 0.168645734917304
 0.1090474396670803
 0.10093276583564408
 0.11288085589914773
 0.05766901929390376
 0.05798324647439323
 0.0642519506681571

## Sequential Quadratic Programming
Remember that SQP is just Newton's method on the Lagrangian. 
Let's recall our update rule for Newton's method:
if $f: \mathbb R^n \to \mathbb R$, and we want to maximize $f$, we start with $x_0$ and update according to 
$$ D^2f(x_k) (x_{k+1} - x_{k}) = -Df(x_k) $$

In [10]:
function newton(f, x0; tol = 1e-8, itermax = 100, trace=false)
    x    = copy(x0)
    err  = Inf
    iter = 0
    while ! (err < tol) && iter < itermax

        # Calculate the gradient and the Hessian
        fx  = f(x)
        Df   = gradient(f, x)
        D²f  = hessian(f, x)

        # search direction
        sk = -D²f\Df

        # update x
        x′ = x + sk

        # Check for convergence
        err = norm(x′ - x)/max(norm(x),1)

        # copy x′ to be old x and start again
        x .= x′
        iter += 1

        trace && @info "Trace Information" iter x fx = f(x)
    end

    return (;fx = f(x), x, iter)
end

newton (generic function with 1 method)

In [19]:
# Setting up the Lagrangian
L(x, λ, α, p) = u(x, α) - λ * (dot(x, p) - 1)

# Let's solve the problem:
function h(z)
    # we need a domain transformation to ensure we're only searching over positive numbers for x
    t = as(Array, as_positive_real, 11)
    y = transform(t, z)

    # separate x and λ and call the Lagrangian
    L(y[1:end-1], y[end], α, p)
end
ret = newton(h, ones(11))

(fx = -0.7531257099977997, x = [-2.5621265795030403, -1.1814427295803822, -0.2961920258578519, -0.23259450675866966, 2.4482446887778244, -1.1298074277450192, -0.1524034471189933, -2.628294560937256, -2.7192404969250115, -1.987217713423624, -1.7762675146586297e-16], iter = 10)

In [20]:
## We still need to get our solution back out:

# Apply the domain transformation again to get 
ys = transform(as(Array, as_positive_real, 11), ret.x)
xs = ys[1:end-1]

# Compare to the true solution
@assert xs ≈ α./p
hcat(xs, α./p)


10×2 Matrix{Float64}:
  0.0771405   0.0771405
  0.306836    0.306836
  0.743645    0.743645
  0.792475    0.792475
 11.568      11.568
  0.323095    0.323095
  0.858642    0.858642
  0.0722015   0.0722015
  0.0659248   0.0659248
  0.137076    0.137076

## Penalty Method
Now, we're going to try an approach with a penalty function.  We will keep using our Newton's method implementation on the inside.

Recall that the penalty method approach solves a sequence of problems:
\begin{equation}
    \max_{x \in \mathbb R^n} f(x) - P_k \sum_{i=1}^m g_i(x)^2
\end{equation}

We need a function that takes in the objective, our constraints, and a penalty value $P_k$.  It should return the solved value.

In [27]:
function solve_penalty(P, x0)
    # Still need the domain transform
    t = as(Array, as_positive_real, length(x0))
    
    # Set up the penalized objective 
    function objective(y)
        x  = transform(t, y) 
        
        return u(x, α) - P * (dot(p, x) - 1)^2
    end

    # Solve with Newton's method
    y0  = inverse(t, x0)
    ret = newton(objective, y0)

    # Keep things interpretable
    ret = (; ret..., x = transform(t, ret.x))
end

solve_penalty (generic function with 1 method)

First, let's observe that if you start with a very large value of $P$, the poor conditioning of the problem shows up.  We get completely incorrect answers!!

In [36]:
ret = solve_penalty(1e20, ones(10))
hcat(ret.x, α./p)


10×2 Matrix{Float64}:
 0.225656   0.0771405
 0.225194   0.306836
 0.231865   0.743645
 0.221432   0.792475
 0.213316  11.568
 0.223461   0.323095
 0.221471   0.858642
 0.223109   0.0722015
 0.224523   0.0659248
 0.224367   0.137076

And if we happen to start with a $P$ that's large, but not _too_ large, it still doesn't save us.  
We get the right answer, but look at how many iterations it takes!

In [40]:
ret = solve_penalty(1e12, ones(10))
@info "We took a lot of iterations to get here" iter = ret.iter

# Compare to the truth
hcat(ret.x, α./p)


┌ Info: We took a lot of iterations to get here
│   iter = 72
└ @ Main /Users/jacobadenbaum/Dropbox/Teaching/2023/Programming for Economics/Lectures/Optimization/tutorial6.ipynb:2


10×2 Matrix{Float64}:
  0.0771405   0.0771405
  0.306836    0.306836
  0.743645    0.743645
  0.792475    0.792475
 11.568      11.568
  0.323095    0.323095
  0.858642    0.858642
  0.0722015   0.0722015
  0.0659248   0.0659248
  0.137076    0.137076

Let's try instead to solve the problem with a sequence of penalty values

In [46]:
function penalty_method(x0; tol = 1e-12, γ = 10, trace = false)

    iter = 0
    num_evals = 0
    P = 1
    x = copy(x0)
    while true
        # Solve with penalty parameter p
        ret = solve_penalty(P, x)

        # Keep track of the total number of function evals (really, Newton steps in this case)
        num_evals += ret.iter

        # Check whether the constraints are violated
        x  = ret.x
        gx = (dot(p, x) - 1)

        # If gx is small enough, we're done
        gx < tol && break

        # Otherwise we keep going, using the solution as a warmstart
        # Increment P up
        x .= ret.x
        P *= γ
        iter += 1

        trace && @info "Trace" iter P penalty_violation = gx
    end

    return (; fx = u(x, α), x, iter, num_evals)
end

penalty_ret = penalty_method(ones(10), trace=true)

┌ Info: Trace
│   iter = 1
│   P = 10
│   penalty_violation = 0.3660254037844386
└ @ Main /Users/jacobadenbaum/Dropbox/Teaching/2023/Programming for Economics/Lectures/Optimization/tutorial6.ipynb:27
┌ Info: Trace
│   iter = 2
│   P = 100
│   penalty_violation = 0.047722557505166296
└ @ Main /Users/jacobadenbaum/Dropbox/Teaching/2023/Programming for Economics/Lectures/Optimization/tutorial6.ipynb:27
┌ Info: Trace
│   iter = 3
│   P = 1000
│   penalty_violation = 0.004975246918103915
└ @ Main /Users/jacobadenbaum/Dropbox/Teaching/2023/Programming for Economics/Lectures/Optimization/tutorial6.ipynb:27
┌ Info: Trace
│   iter = 4
│   P = 10000
│   penalty_violation = 0.000499750249687958
└ @ Main /Users/jacobadenbaum/Dropbox/Teaching/2023/Programming for Economics/Lectures/Optimization/tutorial6.ipynb:27
┌ Info: Trace
│   iter = 5
│   P = 100000
│   penalty_violation = 4.99975002494768e-5
└ @ Main /Users/jacobadenbaum/Dropbox/Teaching/2023/Programming for Economics/Lectures/Optimization/tu

(fx = -0.7531257099972999, x = [0.07714052044215622, 0.3068357381174742, 0.7436446153139314, 0.7924748515785076, 11.56802340627432, 0.3230954696551163, 0.8586417942936179, 0.07220149254441155, 0.06592480550955669, 0.137076280871644], iter = 12, num_evals = 37)

Let's check that we got the right answer!

In [47]:
hcat(penalty_ret.x, α./p)

10×2 Matrix{Float64}:
  0.0771405   0.0771405
  0.306836    0.306836
  0.743645    0.743645
  0.792475    0.792475
 11.568      11.568
  0.323095    0.323095
  0.858642    0.858642
  0.0722015   0.0722015
  0.0659248   0.0659248
  0.137076    0.137076

## Augmented Lagrangian Approach
Let's finally try to implement this using the augmented lagrangian method.
Recall that now we solve a sequence of problems:
\begin{equation}
    \max_{x \in \mathbb \R^n} f(x) - {P_k \over 2} \sum_{i=1}^m g_i(x)^2 - \sum_{i=1}^m \lambda_i^k g_i(x)
\end{equation}
and where we update $\lambda$ according to 
$$ \lambda_i^{k+1} = \lambda_i^k + P_k g_i(x_k) $$

In [58]:
# First, we need to solve the inner augmented lagrangian problem
# we'll use newton's method again
function augmented_lagrangian_inner(f, g, P, λ, x0)

    function objective(x)
        gx = g(x)
        f(x) - 1/2 * P * dot(gx, gx) - dot(λ, gx)
    end

    # Solve the augmented problem
    ret = newton(objective, x0)

    return ret
end

function augmented_lagrangian(f, g, x0; tol = 1e-8, γ = 10, trace=false)

    # setup 
    gx = g(x0)
    λ  = zeros(size(gx))
    P  = 1.0
    x  = copy(x0)
    num_evals = 0
    iter = 0

    while true 
        
        # Solve the inner problem 
        ret = augmented_lagrangian_inner(f, g, P, λ, x)
        x′  = ret.x 
        num_evals += ret.iter
        iter += 1
        # Update λ
        gx  = g(x′)
        λ′  = gx .* P

        # Check if we're violating the constraints
        err = dot(g(x), g(x))
        if err < tol
            break
        else # otherwise keep going
            P *= γ
            x .= x′
            λ .= λ′
            trace && @info "Trace" iter P err penalty_violation = gx
        end
    end

    return (; fx = f(x), x, λ, num_evals, iter)

end

augmented_lagrangian (generic function with 1 method)

In [61]:
# Let's apply now 
function obj(y) 
    t = as(Array, as_positive_real, 10)
    x = transform(t, y)
    return u(x, α)
end

function constraints(y)
    t = as(Array, as_positive_real, 10)
    x = transform(t, y)
    return dot(x, p) - 1
end

ret = augmented_lagrangian(obj, constraints, ones(10), trace=true)
@show ret

ret = (fx = -0.7530611550166022, x = [-2.5620620245218433, -1.1813781745991847, -0.29612747087665425, -0.23252995177747215, 2.4483092437590224, -1.1297428727638217, -0.15233889213779583, -2.6282300059560586, -2.7191759419438135, -1.9871531584424265], λ = fill(0.6455706491514945), num_evals = 29, iter = 6)


┌ Info: Trace
│   iter = 1
│   P = 10.0
│   err = 123.44545286449578
│   penalty_violation = 0.6180339887498947
└ @ Main /Users/jacobadenbaum/Dropbox/Teaching/2023/Programming for Economics/Lectures/Optimization/tutorial6.ipynb:45
┌ Info: Trace
│   iter = 2
│   P = 100.0
│   err = 0.38196601125010493
│   penalty_violation = 0.03483075993763296
└ @ Main /Users/jacobadenbaum/Dropbox/Teaching/2023/Programming for Economics/Lectures/Optimization/tutorial6.ipynb:45
┌ Info: Trace
│   iter = 3
│   P = 1000.0
│   err = 0.0012131818378330172
│   penalty_violation = 0.006452809627836453
└ @ Main /Users/jacobadenbaum/Dropbox/Teaching/2023/Programming for Economics/Lectures/Optimization/tutorial6.ipynb:45
┌ Info: Trace
│   iter = 4
│   P = 10000.0
│   err = 4.163875209309883e-5
│   penalty_violation = 0.00035436479794848097
└ @ Main /Users/jacobadenbaum/Dropbox/Teaching/2023/Programming for Economics/Lectures/Optimization/tutorial6.ipynb:45
┌ Info: Trace
│   iter = 5
│   P = 100000.0
│   err = 1.2

(fx = -0.7530611550166022, x = [-2.5620620245218433, -1.1813781745991847, -0.29612747087665425, -0.23252995177747215, 2.4483092437590224, -1.1297428727638217, -0.15233889213779583, -2.6282300059560586, -2.7191759419438135, -1.9871531584424265], λ = fill(0.6455706491514945), num_evals = 29, iter = 6)

In [60]:
# Compare values
t = as(Array, as_positive_real, 10)
x = transform(t, ret.x)

hcat(x, α./p)

10×2 Matrix{Float64}:
  0.0771455   0.0771405
  0.306856    0.306836
  0.743693    0.743645
  0.792526    0.792475
 11.5688     11.568
  0.323116    0.323095
  0.858697    0.858642
  0.0722062   0.0722015
  0.0659291   0.0659248
  0.137085    0.137076